## Import Libraries

In [4]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict
import torch
import numpy as np
import evaluate
import evaluate
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments
from evaluate import load


## Install Dependencies

In [3]:
!pip install transformers datasets evaluate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00


## Load Dataset

In [10]:
train_file = "/content/pqa_train.json"
test_file = "/content/pqa_test.json"

dataset = load_dataset("json", data_files={"train": train_file, "test": test_file})

print(dataset)
print(dataset["train"][0])


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['data'],
        num_rows: 901
    })
    test: Dataset({
        features: ['data'],
        num_rows: 93
    })
})
{'data': {'paragraphs': [{'context': 'شرکت فولاد مبارکۀ اصفهان، بزرگ\u200cترین واحد صنعتی خصوصی در ایران و بزرگ\u200cترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم\u200cاکنون محرک بسیاری از صنایع بالادستی و پایین\u200cدستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده\u200cاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستین\u200cبار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ\u200cترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شده\u200cاست. مصرف آب این کارخانه در کمترین میزان خود، ۱

## Dataset format

In [11]:
def extract_examples(dataset_split):
    contexts, questions, answers, ids, titles = [], [], [], [], []

    for item in dataset_split["data"]:
        title = item.get("title", "")
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                q_id = qa["id"]
                question = qa["question"]
                ans = qa.get("answers", [])
                if len(ans) > 0:
                    answer_texts = [a["text"] for a in ans]
                    answer_starts = [a["answer_start"] for a in ans]
                else:
                    answer_texts, answer_starts = [], []

                contexts.append(context)
                questions.append(question)
                answers.append({"text": answer_texts, "answer_start": answer_starts})
                ids.append(q_id)
                titles.append(title)

    return Dataset.from_dict({
        "id": ids,
        "title": titles,
        "context": contexts,
        "question": questions,
        "answers": answers
    })

train_dataset = extract_examples(dataset["train"])
test_dataset = extract_examples(dataset["test"])

pquad = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

print(pquad)
print(pquad["train"][0])


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 9008
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 930
    })
})
{'id': 1, 'title': 'شرکت فولاد مبارکه اصفهان', 'context': 'شرکت فولاد مبارکۀ اصفهان، بزرگ\u200cترین واحد صنعتی خصوصی در ایران و بزرگ\u200cترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم\u200cاکنون محرک بسیاری از صنایع بالادستی و پایین\u200cدستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده\u200cاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستین\u200cبار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ\u200cترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر م

In [12]:
import transformers
print(transformers.__version__)

4.55.2


## Load Model, Tokenizer, and Metrics

In [13]:

# Load model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

# Load SQuAD metric
metric = load("squad")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Preprocess Dataset for Training

In [14]:

train_file = "/content/pqa_train.json"
test_file = "/content/pqa_test.json"

dataset = load_dataset("json", data_files={"train": train_file, "test": test_file})

# Inspect dataset
print("Train dataset features:", dataset["train"].features)
print("Test dataset features:", dataset["test"].features)
print("Sample train example:", dataset["train"][0])
print("Sample test example:", dataset["test"][0])

Train dataset features: {'data': {'paragraphs': List({'context': Value('string'), 'qas': List({'answers': List({'answer_end': Value('int64'), 'answer_start': Value('int64'), 'text': Value('string')}), 'id': Value('int64'), 'is_impossible': Value('bool'), 'question': Value('string')})}), 'title': Value('string')}}
Test dataset features: {'data': {'paragraphs': List({'context': Value('string'), 'qas': List({'answers': List({'answer_end': Value('int64'), 'answer_start': Value('int64'), 'text': Value('string')}), 'id': Value('int64'), 'is_impossible': Value('bool'), 'question': Value('string')})}), 'title': Value('string')}}
Sample train example: {'data': {'paragraphs': [{'context': 'شرکت فولاد مبارکۀ اصفهان، بزرگ\u200cترین واحد صنعتی خصوصی در ایران و بزرگ\u200cترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم\u200cاکنون محرک بسیاری از صنایع بالادستی و پایین\u200cدستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در ک

In [15]:

def add_token_positions(example):
    # Tokenize with offsets, attention mask, and token types
    tokenized = tokenizer(
        example["question"],
        example["context"],
        truncation=True,
        max_length=384,
        padding="max_length",
        return_offsets_mapping=True,
        return_attention_mask=True,
        return_token_type_ids=True
    )

    # Get char positions
    if len(example["answers"]["answer_start"]) > 0:
        start_char = example["answers"]["answer_start"][0]
        end_char = start_char + len(example["answers"]["text"][0])
    else:
        start_char, end_char = -1, -1  # For unanswerable questions

    # Map char positions to token positions
    offset_mapping = tokenized["offset_mapping"]
    sequence_ids = tokenized.sequence_ids()

    start_token, end_token = 0, 0  # Default to [CLS] for unanswerable
    if start_char != -1:
        for i, (offset, seq_id) in enumerate(zip(offset_mapping, sequence_ids)):
            if seq_id == 1:  # Context part
                if offset[0] <= start_char < offset[1]:
                    start_token = i
                if offset[0] < end_char <= offset[1]:
                    end_token = i

    tokenized["start_positions"] = start_token
    tokenized["end_positions"] = end_token

    # Preserve id and answers
    tokenized["id"] = example["id"]
    tokenized["answers"] = example["answers"]

    # Remove offset_mapping (as per your original function)
    tokenized.pop("offset_mapping")

    return tokenized

# Apply preprocessing
tokenized_train = pquad["train"].map(add_token_positions, remove_columns=["title", "context", "question"])
tokenized_test = pquad["test"].map(add_token_positions, remove_columns=["title", "context", "question"])

# Verify tokenized dataset
print("Tokenized train keys:", tokenized_train[0].keys())
print("Tokenized test keys:", tokenized_test[0].keys())
print("Sample tokenized test example:", {k: tokenized_test[0][k] for k in ['id', 'answers', 'start_positions', 'end_positions']})

# Check unanswerable questions
unanswerable = [ex for ex in tokenized_test if not ex["answers"]["text"]]
print(f"Number of unanswerable questions in test: {len(unanswerable)}")
if unanswerable:
    print("Sample unanswerable example:", unanswerable[0]["id"], unanswerable[0]["answers"])

Map:   0%|          | 0/9008 [00:00<?, ? examples/s]

Map:   0%|          | 0/930 [00:00<?, ? examples/s]

Tokenized train keys: dict_keys(['id', 'answers', 'input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])
Tokenized test keys: dict_keys(['id', 'answers', 'input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])
Sample tokenized test example: {'id': 9101, 'answers': {'answer_start': [19, 19], 'text': ['مادرید', 'مادرید']}, 'start_positions': 9, 'end_positions': 9}
Number of unanswerable questions in test: 279
Sample unanswerable example: 9115 {'answer_start': [], 'text': []}


## Define evaluation functions

In [16]:
def postprocess_qa_predictions(examples, features, raw_predictions, tokenizer, n_best_size=20, max_answer_length=30):
    all_start_logits, all_end_logits = raw_predictions
    predictions = {}

    for i, feature in enumerate(features):
        example_id = examples[i]["id"]
        start_logits = all_start_logits[i]
        end_logits = all_end_logits[i]

        # Get top-N start and end logits
        start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
        end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

        valid_answers = []
        for start_index in start_indexes:
            for end_index in end_indexes:
                if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                    continue
                if start_index >= len(feature["input_ids"]) or end_index >= len(feature["input_ids"]):
                    continue
                score = start_logits[start_index] + end_logits[end_index]
                answer_ids = feature["input_ids"][start_index:end_index + 1]
                answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
                valid_answers.append({"score": score, "text": answer})

        if valid_answers:
            best_answer = max(valid_answers, key=lambda x: x["score"])
            predictions[example_id] = best_answer["text"]
        else:
            predictions[example_id] = ""  # Empty string for unanswerable

    return predictions



def compute_metrics(p):
    metric = load("squad")

    # Get predictions
    predictions = postprocess_qa_predictions(
        tokenized_test, tokenized_test, p.predictions, tokenizer
    )

    # Convert predictions to correct format with string IDs
    formatted_predictions = [
        {"id": str(k), "prediction_text": v} for k, v in predictions.items()
    ]

    # Convert references to correct format with string IDs
    formatted_references = []
    for ex in tokenized_test:
        answers = ex["answers"]
        # Handle unanswerable questions
        if not answers["text"] or not answers["answer_start"]:
            answers = {"text": [""], "answer_start": [0]}
        formatted_references.append({"id": str(ex["id"]), "answers": answers})

    # Compute metrics
    return metric.compute(predictions=formatted_predictions, references=formatted_references)


# Test compute_metrics with dummy data
dummy_start_logits = np.zeros((len(tokenized_test), 384))  # Match max_length
dummy_end_logits = np.zeros((len(tokenized_test), 384))
dummy_start_logits[:, 0] = 1
dummy_end_logits[:, 1] = 1
dummy_predictions = (dummy_start_logits, dummy_end_logits)
# Create mock EvalPrediction object
metrics = compute_metrics(type('EvalPrediction', (), {'predictions': dummy_predictions})())
print("Dummy metrics:", metrics)

Dummy metrics: {'exact_match': 0.0, 'f1': 1.8729472217437855}


## Set up and run training

In [19]:

training_args = TrainingArguments(
    output_dir="./parsbert-pquad",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
print("Training finished!")

/tmp/ipython-input-37935098.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Exact Match,F1
1,2.253400,1.766209,37.849462,34.925462
2,1.196800,1.840587,38.172043,35.785600


Training finished!


## Evaluate, Predict, and Save Model

In [20]:
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

predictions = trainer.predict(tokenized_test)
preds = postprocess_qa_predictions(tokenized_test, tokenized_test, predictions.predictions, tokenizer)


# Save model
trainer.save_model("./parsbert-pquad-best")
tokenizer.save_pretrained("./parsbert-pquad-best")

Evaluation results: {'eval_loss': 1.8405866622924805, 'eval_exact_match': 38.17204301075269, 'eval_f1': 35.78560037865217, 'eval_runtime': 27.3852, 'eval_samples_per_second': 33.96, 'eval_steps_per_second': 2.154, 'epoch': 2.0}


('./parsbert-pquad-best/tokenizer_config.json',
 './parsbert-pquad-best/special_tokens_map.json',
 './parsbert-pquad-best/vocab.txt',
 './parsbert-pquad-best/added_tokens.json',
 './parsbert-pquad-best/tokenizer.json')